# Νευρωνικά Δίκτυα και Ευφυή Υπολογιστικά Συστήματα
## Εργασία 1: Επιβλεπόμενη Μάθηση - Ταξινόμηση
### Ομάδα 44
* Βασιλάκος Μιχαήλ <br>
Α.Μ.: 03117069
* Γαβαλάς Παντελεήμων <br>
Α.Μ.: 03117075

In [1]:
# import used modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Μέρος 1: UCI dataset
Το dataset του UCI Machine Learning Repository που αντιστοιχεί στην ομάδα μας είναι το [HCC Survival Data Set](https://archive.ics.uci.edu/ml/datasets/HCC+Survival). Το συγκεκριμένο dataset περιέχει πληροφορίες σχετικά με την επιβίωση ασθενών με Ηπατοκυτταρικό καρκίνωμα (HCC) σε διάστημα ενός χρόνου από τη συλλογή των στοιχείων που δίνονται στο dataset. Δίνονται 49 features, 23 εκ των οποίων είναι κατηγορικά ενώ τα υπόλοιπα 26 είναι διατεταγμένα.

Αρχικά φορτώνουμε το dataset στο notebook και ελέγχουμε τα πρώτα στοιχεία του καθώς και τις διαστάσεις του.

In [26]:
# read dataset
df = pd.read_csv("hcc-data.txt", header=None)
print(df.head(), df.shape)

   0  1   2  3  4  5  6   7  8   ...   41    42 43    44   45  46  47  48 49
0   1  0   1  0  0  0  0   1  0  ...  7.1   0.7  1   3.5  0.5   ?   ?   ?  1
1   0  ?   0  0  0  0  1   1  ?  ...    ?     ?  1   1.8    ?   ?   ?   ?  1
2   1  0   1  1  0  1  0   1  0  ...    7   2.1  5    13  0.1  28   6  16  1
3   1  1   1  0  0  0  0   1  0  ...  8.1  1.11  2  15.7  0.2   ?   ?   ?  0
4   1  1   1  1  0  1  0   1  0  ...  6.9   1.8  1     9    ?  59  15  22  1

[5 rows x 50 columns] (165, 50)


Το αρχείο text δεν χρειάστηκε κάποια μετατροπή για να εισαχθεί στο notebook. Βλέπουμε πως υπάρχουν 50 στήλες στο dataframe, το οποίο σημαίνει πως έχουμε 49 χαρακτηριστικά και 1 ετικέτα. Από την περιγραφή των χαρακτηριστικών στο UCI Repository βλέπουμε πως τα πρώτα 23 χαρακτηριστικά είναι μη διατεταγμένα (κατηγορικά) και είναι:
* Gender (1: Άνδρας; 0: Γυναίκα)
* Symptoms (1: Ναι; 0: Όχι)
* Alcohol (1: Ναι; 0: Όχι)
* Hepatitis B Surface Antigen (1: Ναι; 0: Όχι)
* Hepatitis B e Antigen (1: Ναι; 0: Όχι)
* Hepatitis B Core Antibody (1: Ναι; 0: Όχι)
* Hepatitis C Virus Antibody (1: Ναι; 0: Όχι)
* Cirrhosis (1: Ναι; 0: Όχι)
* Endemic Countries (1: Ναι; 0: Όχι)
* Smoking (1: Ναι; 0: Όχι)
* Diabetes (1: Ναι; 0: Όχι)
* Obesity (1: Ναι; 0: Όχι)
* Hemochromatosis (1: Ναι; 0: Όχι)
* Arterial Hypertension (1: Ναι; 0: Όχι)
* Chronic Renal Insufficiency (1: Ναι; 0: Όχι)
* Human Immunodeficiency Virus (1: Ναι; 0: Όχι)
* Nonalcoholic Steatohepatitis (1: Ναι; 0: Όχι)
* Esophageal Varices (1: Ναι; 0: Όχι)
* Splenomegaly (1: Ναι; 0: Όχι)
* Portal Hypertension (1: Ναι; 0: Όχι)
* Portal Vein Thrombosis (1: Ναι; 0: Όχι)
* Liver Metastasis (1: Ναι; 0: Όχι)
* Radiological Hallmark  (1: Ναι; 0: Όχι)

Το dataset δεν περιέχει επικεφαλίδες ούτε αρίθμηση σειρών. Οι ετικέτες των κλάσεων είναι στην 50η στήλη και λαμβάνουν τιμές 1 και 0, όπου το 1 εκφράζει την επιβίωση του ασθενή στον 1 χρόνο ενώ το 0 το αντίθετο.

In [52]:
# missing values are marked with a '?'
df.replace('?', np.nan, inplace=True)
# find missing values
print("Total missing values:", df.isnull().sum(axis=1).sum())
# find samples with missing values
missing_samples = df.isnull().any(axis=1).sum()
print("Number of samples with missing values: {} which represents {:.2f}% \
of total samples".format(missing_samples, missing_samples/df.shape[0]*100))
# find features with missing values
missing_cols = df.isnull().sum(axis=0)
missing_cols_percent = ["{:.1f}%".format(x/df.shape[0]*100) for x in missing_cols]
print("Percentage of samples with missing features:")
for i in range(len(missing_cols_percent)-1):
  print("Feature {}: {}".format(i, missing_cols_percent[i]))

Total missing values: 826
Number of samples with missing values: 157 which represents 95.15% of total samples
Percentage of samples with missing features:
Feature 0: 0.0%
Feature 1: 10.9%
Feature 2: 0.0%
Feature 3: 10.3%
Feature 4: 23.6%
Feature 5: 14.5%
Feature 6: 5.5%
Feature 7: 0.0%
Feature 8: 23.6%
Feature 9: 24.8%
Feature 10: 1.8%
Feature 11: 6.1%
Feature 12: 13.9%
Feature 13: 1.8%
Feature 14: 1.2%
Feature 15: 8.5%
Feature 16: 13.3%
Feature 17: 31.5%
Feature 18: 9.1%
Feature 19: 6.7%
Feature 20: 1.8%
Feature 21: 2.4%
Feature 22: 1.2%
Feature 23: 0.0%
Feature 24: 29.1%
Feature 25: 32.1%
Feature 26: 0.0%
Feature 27: 0.6%
Feature 28: 1.2%
Feature 29: 2.4%
Feature 30: 4.8%
Feature 31: 1.8%
Feature 32: 1.8%
Feature 33: 1.8%
Feature 34: 1.8%
Feature 35: 3.6%
Feature 36: 3.0%
Feature 37: 2.4%
Feature 38: 1.8%
Feature 39: 1.8%
Feature 40: 1.8%
Feature 41: 6.7%
Feature 42: 4.2%
Feature 43: 1.2%
Feature 44: 12.1%
Feature 45: 26.7%
Feature 46: 47.9%
Feature 47: 48.5%
Feature 48: 48.5%


Παρατηρούμε πως μας λείπουν 826 τιμές συνολικά. Δεδομένου ότι έχουμε 49 χαρακτηριστικά και 165 δείγματα μας λείπουν $\frac{826}{49*165}*100\% = 10.2\%$ των τιμών του dataset. Επιπλέον, είδαμε πως πάνω από το 95% των δειγμάτων έχουν τουλάχιστον μία τιμή που απουσιάζει, ενώ επίσης είδαμε πως 7 από τα 49 χαρακτηριστικά απουσιάζουν για πάνω από το 25% των δειγμάτων, ενώ 3 εξ αυτών απουσιάζουν για περίπου τα μισά δείγματα.

In [59]:
# count instances of each class
class1 = df.iloc[:,49].sum()
class0 = df.shape[0] - class1
print("Class 0 has {} samples which represents {:.1f}% of all samples".format
      (class0, class0/df.shape[0]*100))
print("Class 1 has {} samples which represents {:.1f}% of all samples".format
      (class1, class1/df.shape[0]*100))

Class 0 has 63 samples which represents 38.2% of all samples
Class 1 has 102 samples which represents 61.8% of all samples


Παρατηρούμε πως η κλάση 1 (ο ασθενής ζει) αντιπροσωπεύει το 61.8% των δειγμάτων του dataset, έναντι του 38.2% που αντιστοιχεί στην κλάση 0 (ο ασθενής πεθαίνει). Αυτό σημαίνει πως το dataset δεν είναι ισορροπημένο και συνεπώς απαιτείται ειδικός χειρισμός για την αποτελεσματικότερη χρήση του.